# Classification

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/use-cases/classification.ipynb)

In [ ]:
# Supprimer les commentaires pour installer

In [5]:
from utils import test
test()

1